# Project 2: Extract, Transform, and Load
### Team 3: Chris Schultz and Glen Dagger

Quick Links (for use in Jupyter Notebook only)
- [Extract and Transform](#extract-and-transform-data)
  - [Census Data](#census-data)
    - [by County](#by-county)
    - [by State](#by-state)
  - [Beers](#beers)
  - [Breweries](#breweries)
    - [Find and Fix Mismatched Counties](#find-mismatched-counties)
- [Load](#load)
  - [Connect to DB](#connect-to-local-database)
  - [Write to DB](#write-tables-to-postgresql-database)

In [1]:
# Import dependencies
import pandas as pd
from census import Census
from sqlalchemy import create_engine, inspect
import time

# Import county_web_scrape.py for breweries section
import county_web_scrape
from config import api_key, postgres_password, postgres_username

# Extract and Transform Data

## Census Data

### By County

In [2]:
# Create Census object with Census API key from the selected year
c = Census(api_key, year=2020)

# Run Census Search to retrieve income data by county
county_census_data = c.acs5.get(("NAME", "B01003_001E", "B19013_001E","B19301_001E", "B01002_001E"), {'for': 'county:*'})

# Convert to DataFrame
county_census_df = pd.DataFrame(county_census_data)

# Rename columns
county_census_cleaned_df = county_census_df.rename(columns={"B01003_001E": "population",
                                      "B19013_001E": "med_household_income",
                                      "B19301_001E": "per_capita_income",
                                      "B01002_001E": "median_age",
                                      "state": "state_code",
                                      "NAME": "county",
                                      "county": "county_code"
                                    }
                                    )

# Cast population column as integer
county_census_cleaned_df['population'] = county_census_cleaned_df['population'].astype(int)

# Split county column into separate county and state columns
county_census_cleaned_df[['county','state']] = county_census_cleaned_df.county.str.split(', ', expand=True)

# Display first 5 rows
county_census_cleaned_df.head()


,county,population,med_household_income,per_capita_income,median_age,state_code,county_code,state
0,Autauga County,55639,57982.0,29804.0,38.6,01,001,Alabama
1,Baldwin County,218289,61756.0,33751.0,43.2,01,003,Alabama
2,Barbour County,25026,34990.0,20074.0,40.1,01,005,Alabama
3,Bibb County,22374,51721.0,22626.0,39.9,01,007,Alabama
4,Blount County,57755,48922.0,25457.0,41.0,01,009,Alabama


In [3]:
# Create dictionary for converting state names to abbreviations
state_abbreviations = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [4]:
# Convert State names to state abbreviations
county_census_cleaned_df.replace({"state": state_abbreviations}, inplace=True)

# Drop state_code and county_code columns
county_census_final_df = county_census_cleaned_df.drop(columns=['county_code', 'state_code'])

# Export to csv
county_census_final_df.to_csv('./CleanedCSVs/county_census_data_cleaned.csv', index=False)

# Display first 5 rows
county_census_cleaned_df.head()

,county,population,med_household_income,per_capita_income,median_age,state_code,county_code,state
0,Autauga County,55639,57982.0,29804.0,38.6,01,001,AL
1,Baldwin County,218289,61756.0,33751.0,43.2,01,003,AL
2,Barbour County,25026,34990.0,20074.0,40.1,01,005,AL
3,Bibb County,22374,51721.0,22626.0,39.9,01,007,AL
4,Blount County,57755,48922.0,25457.0,41.0,01,009,AL


### By State

In [5]:
# Create Census object with Census API key from the selected year
c = Census(api_key, year=2020)

# Run Census Search to retrieve income data by county
state_census_data = c.acs5.get(("NAME", "B01003_001E", "B19013_001E","B19301_001E", "B01002_001E"), {'for': 'state:*'})

# Convert to DataFrame
state_census_df = pd.DataFrame(state_census_data)

# Rename columns
state_census_cleaned_df = state_census_df.rename(columns={"B01003_001E": "population",
                                      "B19013_001E": "med_household_income",
                                      "B19301_001E": "per_capita_income",
                                      "B01002_001E": "median_age"
                                    }
                                    )

# Cast population column as int
state_census_cleaned_df['population'] = state_census_cleaned_df['population'].astype(int)

# Display first 5 rows
state_census_cleaned_df.head()

,NAME,population,med_household_income,per_capita_income,median_age,state
0,Pennsylvania,12794885,63627.0,35518.0,40.9,42
1,California,39346023,78672.0,38576.0,36.7,06
2,West Virginia,1807426,48037.0,27346.0,42.7,54
3,Utah,3151239,74197.0,30986.0,31.1,49
4,New York,19514849,71117.0,40898.0,39.0,36


In [6]:
# Convert State names to state abbreviations
state_census_cleaned_df.replace({"NAME": state_abbreviations}, inplace=True)

# Drop state_code and county_code columns
state_census_cleaned_df.drop(columns='state', inplace=True)

# Rename 'NAME' column as 'state'
state_census_cleaned_df = state_census_cleaned_df.rename(columns={'NAME': 'state'})

# Export dataframe to CSV to more easily inspect final dataset
state_census_cleaned_df.to_csv('./CleanedCSVs/state_census_data_cleaned.csv')

# Display first 5 rows
state_census_cleaned_df.head()

,state,population,med_household_income,per_capita_income,median_age
0,PA,12794885,63627.0,35518.0,40.9
1,CA,39346023,78672.0,38576.0,36.7
2,WV,1807426,48037.0,27346.0,42.7
3,UT,3151239,74197.0,30986.0,31.1
4,NY,19514849,71117.0,40898.0,39.0


## Beers

In [7]:
# Import beers.csv as dataframe
beers_df = pd.read_csv('./Resources/beers.csv', index_col=[0])

# Filter dataframe to desired columns
beers_cleaned_df = beers_df[['id','name','style','brewery_id','abv']]

# Export to csv
beers_cleaned_df.to_csv('./CleanedCSVs/beers_cleaned.csv', index=False)

# Display first 5 rows
beers_cleaned_df.head()

,id,name,style,brewery_id,abv
0,1436,Pub Beer,American Pale Lager,408,0.050
1,2265,Devil's Cup,American Pale Ale (APA),177,0.066
2,2264,Rise of the Phoenix,American IPA,177,0.071
3,2263,Sinister,American Double / Imperial IPA,177,0.090
4,2262,Sex and Candy,American IPA,177,0.075


## Breweries

In [8]:
# Import breweries.csv as dataframe
breweries_df = pd.read_csv('./Resources/breweries.csv')

# Strip whitespace from state column
breweries_df['state'] = breweries_df['state'].str.strip()

# Strip whitespace from name column
breweries_df['name'] = breweries_df['name'].str.strip()

# Rename brewery_id column
breweries_df.columns = ['brewery_id', 'name', 'city', 'state']

# Export to csv for webscraping
breweries_df.to_csv('./CleanedCSVs/breweries_table.csv')

# Display first 5 rows
breweries_df.head()

,brewery_id,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [9]:
# # Run web scraping module from to find county for each city in city_df and add column to dataframe
# # Only need to run once, then results are saved in csv file /Resources/county_df
# # RUN NEXT CELL AS SHORTCUT INSTEAD

# city_df = county_web_scrape.county_scrape()

record 0/401
0.25% complete
Currently scraping Abingdon, VA...
Success! This city is in Washington County.
--------------------------------
record 1/401
0.5% complete
Currently scraping Abita Springs, LA...
Success! This city is in St. Tammany Parish.
--------------------------------
record 2/401
0.75% complete
Currently scraping Ada, MI...


KeyboardInterrupt: 

In [10]:
# Shortcut to retrieve city_df from csv (exported from county_web_scrape.py) after webscraping the first time
city_df = pd.read_csv('./Resources/county_df.csv', index_col=[0])

# Display first 5 rows
city_df.head()

,city,state,county
0,Abingdon,VA,Washington County
1,Abita Springs,LA,St. Tammany Parish
2,Ada,MI,NaN
3,Afton,VA,NaN
4,Airway Heights,WA,Spokane County


In [11]:
# Merge county data into breweries dataframe
breweries_counties_df = breweries_df.merge(city_df, how='left', on=['city','state'])

# Filter columns
breweries_final_df = breweries_counties_df[['brewery_id', 'name', 'city', 'state', 'county']]

# Export cleaned breweries table
breweries_final_df.to_csv('./CleanedCSVs/breweries_cleaned.csv', index=False)

# Display first 5 rows
breweries_final_df.head()

,brewery_id,name,city,state,county
0,0,NorthGate Brewing,Minneapolis,MN,Hennepin County
1,1,Against the Grain Brewery,Louisville,KY,Jefferson County
2,2,Jack's Abby Craft Lagers,Framingham,MA,Middlesex County
3,3,Mike Hess Brewing Company,San Diego,CA,San Diego County
4,4,Fort Point Beer Company,San Francisco,CA,San Francisco County


### Find and Fix Mismatched counties

In [12]:
# Get list of counties from US census to cross-reference
census_counties = county_census_cleaned_df[['county','state']]

# Add new column so specify origin of each row after concat
census_counties.loc[:,"dataset"] = "counties"

# Get list of counties from brewerys dataset to cross-reference
brewery_counties = breweries_final_df[['county','state']]

# Add new column so specify origin of each row after concat
brewery_counties.loc[:,"dataset"] = "breweries"

# Combine two dataframes of counties, drop matching counties/state rows from each when they match, sort by state/county
combined_counties = pd.concat([census_counties, brewery_counties])\
                        .drop_duplicates(subset=['county', 'state'], keep='first')\
                        .sort_values(['state','county'])

# Drop rows with null values ()
combined_counties.dropna(inplace=True)

# Filter down to scraped county names that do not match census
counties_to_fix = combined_counties[combined_counties['dataset']=='breweries']

# Display mismatched county names
print(counties_to_fix)

                                      county state    dataset
102                        Anchorage Borough    AK  breweries
270                           Juneau Borough    AK  breweries
227              District of Columbia County    DC  breweries
71                     Baltimore City County    MD  breweries
237       Lexington City City (County Equiv)    VA  breweries
115         Norfolk City City (County Equiv)    VA  breweries
343        Richmond City City (County Equiv)    VA  breweries
427         Roanoke city City (County Equiv)    VA  breweries
306  Virginia Beach City City (County Equiv)    VA  breweries


/Users/glena.dagger/Downloads/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/glena.dagger/Downloads/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
# Create dictionary to change mismatched county names in brewery table to match county census data
county_name_fix = {'Anchorage Borough': 'Anchorage Municipality',
                    'Juneau Borough': 'Juneau City and Borough',
                    'District of Columbia County': 'District of Columbia',
                    'Baltimore City County': 'Baltimore city',
                    'Lexington City City (County Equiv)': 'Lexington city',
                    'Norfolk City City (County Equiv)': 'Norfolk city',
                    'Richmond City City (County Equiv)': 'Richmond city',
                    'Roanoke city City (County Equiv)': 'Roanoke city',
                    'Virginia Beach City City (County Equiv)': 'Virginia Beach city'
                    }

breweries_final_df.replace({'county': county_name_fix}, inplace=True)

breweries_final_df

,brewery_id,name,city,state,county
0,0,NorthGate Brewing,Minneapolis,MN,Hennepin County
1,1,Against the Grain Brewery,Louisville,KY,Jefferson County
2,2,Jack's Abby Craft Lagers,Framingham,MA,Middlesex County
3,3,Mike Hess Brewing Company,San Diego,CA,San Diego County
4,4,Fort Point Beer Company,San Francisco,CA,San Francisco County
...,...,...,...,...,...
553,553,Covington Brewhouse,Covington,LA,St. Tammany Parish
554,554,Dave's Brewfarm,Wilson,WI,St. Croix County
555,555,Ukiah Brewing Company,Ukiah,CA,Mendocino County
556,556,Butternuts Beer and Ale,Garrattsville,NY,NaN


# Load

### Connect to PostgreSQL database

In [14]:
# Create connection to postgresql database
protocol = 'postgresql'
username = postgres_username
password = postgres_password
host = 'localhost'
port = 5432
database_name = 'beer_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [15]:
# Check table names using inspector after creating table schema in pgAdmin
inspector = inspect(engine)
table_names = inspector.get_table_names()

table_names

['state_census', 'county_census', 'breweries', 'beers']

### Write data to PostgreSQL database

In [128]:
# Write state census records to postgres database
state_census_cleaned_df.to_sql(name='state_census', con=engine, if_exists='append', index=False)

In [129]:
# Write county census records to postgres database
county_census_final_df.to_sql(name='county_census', con=engine, if_exists='append', index=False)

In [130]:
# Write brewery records to postgres database
breweries_final_df.to_sql(name='breweries', con=engine, if_exists='append', index=False)

In [131]:
# Write beer records to postgres database
beers_cleaned_df.to_sql(name='beers', con=engine, if_exists='append', index=False)